In [1]:
from copy import deepcopy
from tqdm import tqdm
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import plotly.express as px

In [2]:
df = pd.read_csv('data.csv')

In [3]:
RANDOM_STATE = 42

TRAIN_SIZE = 0.6
VAL_SIZE = 0.15
TEST_SIZE = 0.25

# Разбиение выборки

**Разбивать будем случайно во времени** (без отложенного теста), т.к. большая часть выборки сконцентрирована в ноябре 2023 

In [4]:
# проверка уникальности `item_id`
df['item_id'].nunique() == df.shape[0]

True

In [5]:
# отделяем test
train_val_ids, test_ids = train_test_split(
    df['item_id'],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    shuffle=True,
    stratify=df['attr_value_name']
)

# отделяем train
train_ids, val_ids = train_test_split(
    df.loc[df['item_id'].isin(train_val_ids), 'item_id'],
    test_size=VAL_SIZE * (TRAIN_SIZE + VAL_SIZE + TEST_SIZE) / (TRAIN_SIZE + VAL_SIZE),
    random_state=RANDOM_STATE,
    shuffle=True,
    stratify=df.loc[df['item_id'].isin(train_val_ids),'attr_value_name']
)

In [6]:
df.loc[df['item_id'].isin(train_ids), 'sample_part'] = 'train'
df.loc[df['item_id'].isin(val_ids), 'sample_part'] = 'val'
df.loc[df['item_id'].isin(test_ids), 'sample_part'] = 'test'

print(df['sample_part'].value_counts(dropna=False))
print(df['sample_part'].value_counts(dropna=False, normalize=True))

sample_part
train    119606
test      49836
val       29902
Name: count, dtype: int64
sample_part
train    0.599998
test     0.250000
val      0.150002
Name: proportion, dtype: float64


In [7]:
# проверяем как сработала стратификация
for sample_part in df['sample_part'].value_counts(dropna=False).keys():
    mask_sp = df['sample_part'] == sample_part
    print(
        f"""{sample_part}:\n\t{df.loc[mask_sp, 'attr_value_name'].value_counts(dropna=False, normalize=True)}""",
        end='\n\n'
    )

train:
	attr_value_name
kosmeticheskii     0.464592
evro               0.248282
trebuet_remonta    0.190199
dizainerskii       0.096927
Name: proportion, dtype: float64

test:
	attr_value_name
kosmeticheskii     0.464584
evro               0.248274
trebuet_remonta    0.190204
dizainerskii       0.096938
Name: proportion, dtype: float64

val:
	attr_value_name
kosmeticheskii     0.464618
evro               0.248278
trebuet_remonta    0.190188
dizainerskii       0.096917
Name: proportion, dtype: float64



In [8]:
df[['item_id', 'sample_part']].to_csv('splitting_sample.csv', index=False)

# Базелин

**Идея для базелина**: сделать классификацию на основе регулярок. Это noML подход, но он будет давать какую-то базовую точность

# Выделение самых частых слов в описании объявления

In [9]:
df['attr_value_name'].value_counts()

attr_value_name
kosmeticheskii     92614
evro               49493
trebuet_remonta    37915
dizainerskii       19322
Name: count, dtype: int64

In [26]:
russian_stopwords = stopwords.words("russian")
russian_stopwords[:5]

['и', 'в', 'во', 'не', 'что']

In [39]:
count_vect = CountVectorizer(
    encoding='utf-8',
    lowercase=True,
    stop_words=russian_stopwords,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 2),
    analyzer='word',
    max_df=1.0,
    min_df=1
)
tokenizer = count_vect.build_tokenizer()

stemmer = SnowballStemmer('russian') 

text_tokens = [tokenizer(text) for text in tqdm(df['description'].values)]
text_tokens_stem = [
    [stemmer.stem(word) for word in text] for text in tqdm(text_tokens)
    ]
df['description_tokens_stem'] = text_tokens_stem

100%|██████████| 199344/199344 [06:03<00:00, 548.93it/s]


## `trebuet_remonta`

In [75]:
mask_kosm = df['attr_value_name'] == 'trebuet_remonta'

tokens_in_cat = [token for text in df.loc[mask_kosm, 'description_tokens_stem'].values for token in text]
Counter(tokens_in_cat).most_common(100)

[('дом', 78844),
 ('на', 72177),
 ('квартир', 44723),
 ('для', 29723),
 ('по', 28902),
 ('от', 19718),
 ('все', 19525),
 ('детск', 19017),
 ('этаж', 18011),
 ('кв', 17916),
 ('ест', 17596),
 ('не', 16784),
 ('прода', 16730),
 ('район', 16193),
 ('школ', 16128),
 ('ипотек', 15831),
 ('до', 15449),
 ('комнат', 14940),
 ('под', 14189),
 ('магазин', 14144),
 ('из', 14067),
 ('ряд', 13932),
 ('площад', 13907),
 ('сад', 13490),
 ('кухн', 13471),
 ('окн', 13031),
 ('ремонт', 12986),
 ('доступн', 12951),
 ('участк', 12917),
 ('больш', 12188),
 ('продаж', 12085),
 ('минут', 11740),
 ('нов', 11597),
 ('город', 11337),
 ('звон', 11208),
 ('двор', 11059),
 ('отделк', 11051),
 ('без', 10606),
 ('центр', 10479),
 ('участок', 10405),
 ('вод', 10323),
 ('мест', 10302),
 ('инфраструктур', 10248),
 ('жил', 9803),
 ('метр', 9199),
 ('располож', 9024),
 ('можн', 8910),
 ('удобн', 8873),
 ('собственник', 8869),
 ('остановк', 8862),
 ('возможн', 8652),
 ('ваш', 8530),
 ('наход', 8261),
 ('шагов', 8189),
 ('

Можно выделить с помощью:
- `отделк` + `без`

In [55]:
mask = df['description_tokens_stem'].apply(lambda tokens: ('отделк' in tokens) & ('без' in tokens))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['trebuet_remonta']}""")
df.loc[mask, 'attr_value_name'].value_counts(normalize=True)

find 5819 of 37915


attr_value_name
trebuet_remonta    0.517615
kosmeticheskii     0.222719
evro               0.163086
dizainerskii       0.096580
Name: proportion, dtype: float64

## `kosmeticheskii`

In [56]:
mask_kosm = df['attr_value_name'] == 'kosmeticheskii'

tokens_in_cat = [token for text in df.loc[mask_kosm, 'description_tokens_stem'].values for token in text]
Counter(tokens_in_cat).most_common(50)

[('на', 148144),
 ('квартир', 145525),
 ('дом', 132077),
 ('для', 66451),
 ('по', 52966),
 ('все', 51405),
 ('ест', 46032),
 ('детск', 45922),
 ('комнат', 42911),
 ('не', 41862),
 ('школ', 41539),
 ('магазин', 39599),
 ('ряд', 36313),
 ('кухн', 35679),
 ('район', 35046),
 ('ремонт', 34071),
 ('кв', 33956),
 ('доступн', 33836),
 ('этаж', 33815),
 ('от', 33694),
 ('сад', 33532),
 ('окн', 32910),
 ('прода', 32549),
 ('до', 30341),
 ('больш', 29497),
 ('хорош', 29339),
 ('нов', 29010),
 ('двор', 28829),
 ('остановк', 27206),
 ('собственник', 26080),
 ('мебел', 25997),
 ('без', 25248),
 ('шагов', 24189),
 ('ипотек', 24182),
 ('минут', 23967),
 ('тепл', 23811),
 ('продаж', 23689),
 ('из', 23538),
 ('город', 23396),
 ('площад', 23374),
 ('инфраструктур', 22987),
 ('вод', 22741),
 ('центр', 22392),
 ('звон', 22370),
 ('мест', 21196),
 ('чист', 20567),
 ('удобн', 19621),
 ('во', 19382),
 ('можн', 19217),
 ('уютн', 19206)]

Что-то выделяется с помощью:
- `хорош` + `ремонт`

Но довольно неточно

In [62]:
mask = df['description_tokens_stem'].apply(lambda tokens: ('хорош' in tokens) & ('ремонт' in tokens))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['trebuet_remonta']}""")
df.loc[mask, 'attr_value_name'].value_counts(normalize=True)

find 22519 of 37915


attr_value_name
kosmeticheskii     0.435188
evro               0.368489
trebuet_remonta    0.100315
dizainerskii       0.096008
Name: proportion, dtype: float64

## `evro`

In [64]:
mask_kosm = df['attr_value_name'] == 'evro'

tokens_in_cat = [token for text in df.loc[mask_kosm, 'description_tokens_stem'].values for token in text]
Counter(tokens_in_cat).most_common(100)

[('квартир', 99095),
 ('на', 93545),
 ('дом', 68383),
 ('для', 46564),
 ('все', 34812),
 ('детск', 31352),
 ('по', 31129),
 ('ремонт', 30620),
 ('не', 27493),
 ('кухн', 25846),
 ('нов', 25621),
 ('ест', 25579),
 ('комнат', 23971),
 ('школ', 23159),
 ('от', 22034),
 ('магазин', 22009),
 ('мебел', 21552),
 ('до', 21318),
 ('доступн', 21170),
 ('район', 20886),
 ('ряд', 20710),
 ('этаж', 20262),
 ('двор', 19807),
 ('больш', 19532),
 ('кв', 19470),
 ('сад', 18941),
 ('минут', 18457),
 ('техник', 18427),
 ('без', 17257),
 ('окн', 16782),
 ('прода', 16601),
 ('собственник', 16535),
 ('из', 16206),
 ('инфраструктур', 16086),
 ('центр', 15498),
 ('хорош', 15190),
 ('тепл', 14362),
 ('город', 14320),
 ('пол', 14225),
 ('звон', 13952),
 ('площад', 13892),
 ('отличн', 13878),
 ('остановк', 13796),
 ('просторн', 13785),
 ('удобн', 13664),
 ('метр', 13646),
 ('продаж', 13610),
 ('необходим', 13198),
 ('мест', 13131),
 ('шагов', 13102),
 ('площадк', 13076),
 ('уютн', 13015),
 ('ипотек', 12819),
 ('в

Пробуем:
- `тепл` + `пол`
- `хорош` + `ремонт`
- `нов` + `ремонт`
- `современ` + `ремонт`

In [65]:
mask = df['description_tokens_stem'].apply(lambda tokens: ('тепл' in tokens) & ('пол' in tokens))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['evro']}""")
df.loc[mask, 'attr_value_name'].value_counts(normalize=True)

find 19817 of 49493


attr_value_name
evro               0.327749
kosmeticheskii     0.313771
dizainerskii       0.182873
trebuet_remonta    0.175607
Name: proportion, dtype: float64

In [69]:
mask = df['description_tokens_stem'].apply(lambda tokens: ('хорош' in tokens) & ('ремонт' in tokens))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['evro']}""")
df.loc[mask, 'attr_value_name'].value_counts(normalize=True)

find 22519 of 49493


attr_value_name
kosmeticheskii     0.435188
evro               0.368489
trebuet_remonta    0.100315
dizainerskii       0.096008
Name: proportion, dtype: float64

In [66]:
mask = df['description_tokens_stem'].apply(lambda tokens: ('нов' in tokens) & ('ремонт' in tokens))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['evro']}""")
df.loc[mask, 'attr_value_name'].value_counts(normalize=True)

find 27379 of 49493


attr_value_name
evro               0.394280
kosmeticheskii     0.347785
dizainerskii       0.165200
trebuet_remonta    0.092735
Name: proportion, dtype: float64

In [67]:
mask = df['description_tokens_stem'].apply(lambda tokens: ('современ' in tokens) & ('ремонт' in tokens))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['evro']}""")
df.loc[mask, 'attr_value_name'].value_counts(normalize=True)

find 13251 of 49493


attr_value_name
evro               0.501019
kosmeticheskii     0.231605
dizainerskii       0.206400
trebuet_remonta    0.060977
Name: proportion, dtype: float64

1, 3 и 4 комбинации хорошо работают (особенно 4-я)

## `dizainerskii`

In [70]:
mask_kosm = df['attr_value_name'] == 'dizainerskii'

tokens_in_cat = [token for text in df.loc[mask_kosm, 'description_tokens_stem'].values for token in text]
Counter(tokens_in_cat).most_common(100)

[('на', 48267),
 ('квартир', 36260),
 ('дом', 32935),
 ('для', 24159),
 ('все', 16223),
 ('по', 15050),
 ('ремонт', 13274),
 ('кухн', 12446),
 ('не', 12298),
 ('от', 11565),
 ('детск', 11410),
 ('комнат', 11112),
 ('этаж', 11077),
 ('ест', 10805),
 ('из', 10501),
 ('нов', 10346),
 ('до', 10302),
 ('больш', 9364),
 ('мебел', 9357),
 ('спальн', 9155),
 ('техник', 8873),
 ('минут', 8085),
 ('центр', 7806),
 ('доступн', 7796),
 ('кв', 7777),
 ('ряд', 7407),
 ('пол', 7406),
 ('район', 7309),
 ('магазин', 7208),
 ('школ', 7161),
 ('мест', 7037),
 ('гостин', 6875),
 ('территор', 6852),
 ('метр', 6662),
 ('зон', 6627),
 ('тепл', 6589),
 ('сад', 6551),
 ('двор', 6515),
 ('город', 6487),
 ('комплекс', 6485),
 ('прода', 6468),
 ('собственник', 6289),
 ('окн', 6238),
 ('площад', 6235),
 ('просторн', 6209),
 ('без', 6109),
 ('вид', 6076),
 ('инфраструктур', 6053),
 ('под', 5989),
 ('машин', 5964),
 ('звон', 5720),
 ('располож', 5651),
 ('удобн', 5611),
 ('качествен', 5560),
 ('продаж', 5546),
 ('пл

Пробуем выделить:
- `дизайнерск` + `ремонт`

In [71]:
mask = df['description_tokens_stem'].apply(lambda tokens: ('дизайнерск' in tokens) & ('ремонт' in tokens))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['dizainerskii']}""")
df.loc[mask, 'attr_value_name'].value_counts(normalize=True)

find 5860 of 19322


attr_value_name
dizainerskii       0.707167
evro               0.167577
trebuet_remonta    0.063652
kosmeticheskii     0.061604
Name: proportion, dtype: float64

Работает отлично!

# Пробуем найти самые важные фичи TF-IDF

In [228]:
def best_features(model: LogisticRegression, vect: CountVectorizer | TfidfVectorizer, top_n: int = 10, height: int = 1000):
    vocabulary = list(vect.vocabulary_.keys())
    coefs = [[vocabulary[i], abs(coef)] for i, coef in enumerate(model.coef_[0])]
    coefs = sorted(coefs, key=lambda x: x[1], reverse=True)
    data = pd.DataFrame(coefs, columns=['feature', 'coef']).iloc[:top_n]
    fig = px.bar(data, x='coef', y='feature', orientation='h', height=height)
    return fig

In [229]:
bow_vect = CountVectorizer(
    encoding='utf-8',
    lowercase=True,
    stop_words=russian_stopwords,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 2),
    analyzer='word',
    max_df=0.5,
    min_df=1000
)

In [424]:
tokenizer = bow_vect.build_tokenizer()
stemmer = SnowballStemmer('russian') 

text_tokens = [tokenizer(text) for text in tqdm(df['description'].values)]
text_tokens_stem = [
    ' '.join([stemmer.stem(word) for word in text]) for text in tqdm(text_tokens) # применяем стемминг и обратно преобразуем в текст
    ]
df['description_text_stem'] = text_tokens_stem

100%|██████████| 199344/199344 [06:13<00:00, 534.14it/s]


In [230]:
# Применяем bow vectorizer
bow_vect.fit(
    df.loc[df['sample_part'] == 'train', 'description_text_stem']
)
len(bow_vect.vocabulary_)

1751

Учим 4 классификатора (логрега) – один против всех

In [231]:
categories = ['trebuet_remonta', 'kosmeticheskii', 'evro', 'dizainerskii']

ohe = OneHotEncoder(
    categories=[categories],
    sparse_output=False
    )

df.loc[df['sample_part'] == 'train', categories] = ohe.fit_transform(df.loc[df['sample_part'] == 'train', ['attr_value_name']])

### `trebuet_remonta`

In [236]:
pipe = Pipeline(
    steps=[('ss', StandardScaler()),
           ('lr', LogisticRegression(random_state=RANDOM_STATE))]
)

pipe.fit(
    X=bow_vect.transform(df.loc[df['sample_part'] == 'train', 'description_text_stem']).toarray(),
    y=df.loc[df['sample_part'] == 'train', 'trebuet_remonta']
)

Pipeline(steps=[('ss', StandardScaler()),
                ('lr', LogisticRegression(random_state=42))])

In [237]:
best_features(model=pipe['lr'], vect=bow_vect, top_n=50, height=1000)

Ничего сильно хорошего нет

In [291]:
mask = df['description_text_stem'].apply(lambda text: ('входн групп' in text))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['trebuet_remonta']}""")
print(df.loc[mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')
# print(df.loc[~mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')

find 2059 of 37915
attr_value_name
trebuet_remonta    0.298203
evro               0.287518
dizainerskii       0.212725
kosmeticheskii     0.201554
Name: proportion, dtype: float64



### `kosmeticheskii`

In [239]:
pipe = Pipeline(
    steps=[('ss', StandardScaler()),
           ('lr', LogisticRegression(random_state=RANDOM_STATE))]
)

pipe.fit(
    X=bow_vect.transform(df.loc[df['sample_part'] == 'train', 'description_text_stem']).toarray(),
    y=df.loc[df['sample_part'] == 'train', 'kosmeticheskii']
)

best_features(model=pipe['lr'], vect=bow_vect, top_n=50, height=1000)

Получилось найти:
- `окн пластиков`
- `груш`

In [292]:
mask = df['description_text_stem'].apply(lambda text: ('окн пластиков' in text))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['kosmeticheskii']}""")
print(df.loc[mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')
# print(df.loc[~mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')

find 4330 of 92614
attr_value_name
kosmeticheskii     0.644111
trebuet_remonta    0.163972
evro               0.157044
dizainerskii       0.034873
Name: proportion, dtype: float64



In [293]:
mask = df['description_text_stem'].apply(lambda text: ('груш' in text))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['kosmeticheskii']}""")
print(df.loc[mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')
# print(df.loc[~mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')

find 2405 of 92614
attr_value_name
kosmeticheskii     0.587110
trebuet_remonta    0.209148
evro               0.122661
dizainerskii       0.081081
Name: proportion, dtype: float64



### `evro`

In [247]:
pipe = Pipeline(
    steps=[('ss', StandardScaler()),
           ('lr', LogisticRegression(random_state=RANDOM_STATE))]
)

pipe.fit(
    X=bow_vect.transform(df.loc[df['sample_part'] == 'train', 'description_text_stem']).toarray(),
    y=df.loc[df['sample_part'] == 'train', 'evro']
)

best_features(model=pipe['lr'], vect=bow_vect, top_n=50, height=1000)

- `утюг` (что бы это не значило)
- `посудомоечн машин`
- `сплит`

In [294]:
mask = df['description_text_stem'].apply(lambda text: ('утюг' in text))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['evro']}""")
print(df.loc[mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')
# print(df.loc[~mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')

find 2158 of 49493
attr_value_name
evro               0.458295
kosmeticheskii     0.328082
dizainerskii       0.208063
trebuet_remonta    0.005561
Name: proportion, dtype: float64



In [295]:
mask = df['description_text_stem'].apply(lambda text: ('посудомоечн машин' in text))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['evro']}""")
print(df.loc[mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')
# print(df.loc[~mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')

find 4735 of 49493
attr_value_name
evro               0.488490
dizainerskii       0.332418
kosmeticheskii     0.172122
trebuet_remonta    0.006969
Name: proportion, dtype: float64



In [296]:
mask = df['description_text_stem'].apply(lambda text: ('сплит' in text))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['evro']}""")
print(df.loc[mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')
# print(df.loc[~mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')

find 3736 of 49493
attr_value_name
evro               0.408458
kosmeticheskii     0.393201
dizainerskii       0.157388
trebuet_remonta    0.040953
Name: proportion, dtype: float64



### `dizainerskii`

In [263]:
pipe = Pipeline(
    steps=[('ss', StandardScaler()),
           ('lr', LogisticRegression(random_state=RANDOM_STATE))]
)

pipe.fit(
    X=bow_vect.transform(df.loc[df['sample_part'] == 'train', 'description_text_stem']).toarray(),
    y=df.loc[df['sample_part'] == 'train', 'dizainerskii']
)

best_features(model=pipe['lr'], vect=bow_vect, top_n=50, height=1000)

- `натуральн`
- `апартамент`

In [297]:
mask = df['description_text_stem'].apply(lambda text: ('натуральн' in text))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['dizainerskii']}""")
print(df.loc[mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')
# print(df.loc[~mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')

find 1882 of 19322
attr_value_name
dizainerskii       0.387354
evro               0.278427
kosmeticheskii     0.195005
trebuet_remonta    0.139214
Name: proportion, dtype: float64



In [298]:
mask = df['description_text_stem'].apply(lambda text: ('апартамент' in text))
print(f"""find {np.sum(mask)} of {df['attr_value_name'].value_counts()['dizainerskii']}""")
print(df.loc[mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')
# print(df.loc[~mask, 'attr_value_name'].value_counts(normalize=True), end='\n\n')

find 2428 of 19322
attr_value_name
dizainerskii       0.410626
evro               0.254119
trebuet_remonta    0.203871
kosmeticheskii     0.131384
Name: proportion, dtype: float64



# Пробую выделить лучшие n-граммы циклом по всем n-граммам

### `trebuet_remonta`

In [311]:
def find_best_ngrams(target_category: str, vect: CountVectorizer | TfidfVectorizer) -> list:
    mask_train = df['sample_part'] == 'train'

    results = []
    base_ratio = df.loc[mask_train, 'attr_value_name'].value_counts(normalize=True)[target_category]
    for ngram in tqdm(vect.vocabulary_):
        mask_ngram = df['description_text_stem'].apply(lambda text: ngram in text)
        ngram_ratio = df.loc[mask_train & mask_ngram, 'attr_value_name'].value_counts(normalize=True).get(target_category, default=0)
        results.append([ngram, ngram_ratio / base_ratio])

    results = sorted(results, key=lambda x: x[1], reverse=True)
    return results

In [318]:
trebuet_remonta_ngrams = find_best_ngrams(target_category='trebuet_remonta', vect=bow_vect)
kosmeticheskii_ngrams = find_best_ngrams(target_category='kosmeticheskii', vect=bow_vect)
evro_ngrams = find_best_ngrams(target_category='evro', vect=bow_vect)
dizainerskii_ngrams = find_best_ngrams(target_category='dizainerskii', vect=bow_vect)

100%|██████████| 1751/1751 [02:30<00:00, 11.63it/s]


In [323]:
trebuet_remonta_ngrams[:5]

[['отвеч вопрос', 5.257637698360368],
 ['чернов отделк', 4.604007526070034],
 ['чернов', 4.3491179040836965],
 ['квартир треб', 4.199052926878414],
 ['треб ремонт', 4.191095732961809]]

In [324]:
kosmeticheskii_ngrams[:5]

[['ответ ваш', 2.1524258566081196],
 ['сдела косметическ', 1.9431229480811805],
 ['косметическ ремонт', 1.8328146584521179],
 ['косметическ', 1.829706400981503],
 ['пол линолеум', 1.639308856735596]]

In [325]:
evro_ngrams[:5]

[['никт жил', 4.027680495689655],
 ['никт прописа', 4.027680495689655],
 ['евроремонт', 3.364982648705699],
 ['современ ремонт', 2.8296495005351554],
 ['качествен ремонт', 2.340362732851926]]

In [326]:
dizainerskii_ngrams[:5]

[['дизайнерск ремонт', 8.316220960773475],
 ['дизайнерск', 6.443614853027728],
 ['дизайн', 5.368240857427983],
 ['дизайн проект', 5.178308485137517],
 ['необходим жизн', 5.158543948934702]]

Подход не идеален, но на фоне остальных выглядит самым удачным.

Возьмем его за основу

# Определение экспертного правила (на основе регулярок)

### Описание работы baseline:

Будем определять категорию объявления следующим образом:
1) Заводим счетчик баллов категорий как словарь, где значение для каждой категории инициализировано нулем.
2) Для стеммизированного описания объявления пройдемся по всем n-gram с `важностью` (важность – 2-й элемент списков выше) > `threshold`:
    - Если n-gram есть в описании => прибавляем к соотвествующей категории `важность` n-gram'ы.
    - Если нет => ничего не делаем
3) Нормируем на общую важность категории (сумма важностей всех n-gram, прошедших трешхолд)
4) Берем argmax. Т.е. выбираем категорию с наибольшим суммарным баллом.
5) Если у всех категорий по 0 баллов => берем случайную категорию

In [346]:
THRESHOLD = 1.5 # берем н-граммы с силой > 1.5
CATEGORIES = ['trebuet_remonta', 'kosmeticheskii', 'evro', 'dizainerskii']
CAT_PROBABILITIES = df.loc[df['sample_part'] == 'train', 'attr_value_name'].value_counts(normalize=True)

In [398]:
n_grams = {
    'trebuet_remonta': trebuet_remonta_ngrams,
    'kosmeticheskii': kosmeticheskii_ngrams,
    'evro': evro_ngrams,
    'dizainerskii': dizainerskii_ngrams,
}

In [406]:
def filter_n_grams(
        n_grams: dict,
        threshold: float = THRESHOLD,
        categories: list = CATEGORIES
    ) -> dict:
    """Функция для фильтрации n-gram. Берем только n-gram с важностью большей чем трешхолд.
    """
    filtered_n_grams = deepcopy(n_grams) # делаем копию
    for cat in categories:
        filtered_n_grams[cat] = [[ngram, imp] for ngram, imp in n_grams[cat] if imp > threshold] # фильтруем н-граммы
    return filtered_n_grams

In [407]:
def calc_total_points(
        filtered_n_grams: dict,
        categories: list = CATEGORIES
    ) -> dict:
    """Возвращает сумму важностей n-gram в категории.
    """
    total_points = {cat: 0 for cat in categories}
    for cat in categories:
        for ngram, imp in filtered_n_grams[cat]:
            total_points[cat] += imp

    return total_points

In [408]:
def baseline_category(
        stem_description: str,
        filtered_n_grams: dict,
        total_points_cat: dict,
        cat_probabilities: dict = CAT_PROBABILITIES,
        categories: list = CATEGORIES
    ) -> str:
    """Определяем категорию для конкретного объявления.
    """
    points = {cat: 0 for cat in categories}
    for cat in categories:
        for ngram, imp in filtered_n_grams[cat]:
            if ngram in stem_description:
                points[cat] += imp

    points_norm = {
        cat: points[cat] / total_points_cat[cat] for cat in categories
    }
    best_cat = sorted(points_norm.items(), key=lambda x: x[1], reverse=True)[0]
    if best_cat[1] > 0: # есть points > 0
        return best_cat[0] # пользуюсь тем, что словарь отсортирован
    else:
        cat_proba = [cat_probabilities[cat] for cat in categories]
        return np.random.choice(categories, p=cat_proba)

In [394]:
filtered_n_grams_ = filter_n_grams(
    n_grams=n_grams,
    threshold=THRESHOLD,
    categories=CATEGORIES
)

total_points_cat = calc_total_points(
    filtered_n_grams=filtered_n_grams_,
    categories=CATEGORIES
)

np.random.seed(RANDOM_STATE) # обязательно фиксируем seed
df['baseline_prediction'] = df['description_text_stem'].apply(lambda descr: baseline_category(
    stem_description=descr,
    filtered_n_grams=filtered_n_grams_,
    total_points_cat=total_points_cat,
    cat_probabilities=CAT_PROBABILITIES,
    categories=CATEGORIES
    )
)

Проверяем точность

In [384]:
print('baseline accuracy:', (df['baseline_prediction'] == df['attr_value_name']).mean())

baseline accuracy: 0.4108676458784814


### А есть без нормализации...

In [385]:
filtered_n_grams_ = filter_n_grams(
    n_grams=n_grams,
    threshold=THRESHOLD,
    categories=CATEGORIES
)

total_points_cat = {cat: 1 for cat in CATEGORIES} # нормируем на 1

np.random.seed(RANDOM_STATE) # обязательно фиксируем seed
df['baseline_prediction_wo_norm'] = df['description_text_stem'].apply(lambda descr: baseline_category(
    stem_description=descr,
    filtered_n_grams=filtered_n_grams_,
    total_points_cat=total_points_cat,
    cat_probabilities=CAT_PROBABILITIES,
    categories=CATEGORIES
    )
)

In [387]:
print('baseline accuracy wo normalization:', (df['baseline_prediction_wo_norm'] == df['attr_value_name']).mean())

baseline accuracy wo normalization: 0.20066317521470423


Довольно грустно

### Другой трешхолд

Попробуем пошатать трешхолд

#### 1.25

In [411]:
filtered_n_grams_ = filter_n_grams(
    n_grams=n_grams,
    threshold=1.25, # задаем меньший трешхолд
    categories=CATEGORIES
)

total_points_cat = calc_total_points(
    filtered_n_grams=filtered_n_grams_,
    categories=CATEGORIES
)

np.random.seed(RANDOM_STATE) # обязательно фиксируем seed
df['baseline_prediction_1.25'] = df['description_text_stem'].apply(lambda descr: baseline_category(
    stem_description=descr,
    filtered_n_grams=filtered_n_grams_,
    total_points_cat=total_points_cat,
    cat_probabilities=CAT_PROBABILITIES,
    categories=CATEGORIES
    )
)

In [412]:
print('baseline accuracy (threshold 1.25):', (df['baseline_prediction_1.25'] == df['attr_value_name']).mean())

baseline accuracy (threshold 1.25): 0.4288967814431335


Время обработки выросло, но и точность подросла

#### 1.75

In [413]:
filtered_n_grams_ = filter_n_grams(
    n_grams=n_grams,
    threshold=1.75, # задаем больший трешхолд
    categories=CATEGORIES
)

total_points_cat = calc_total_points(
    filtered_n_grams=filtered_n_grams_,
    categories=CATEGORIES
)

np.random.seed(RANDOM_STATE) # обязательно фиксируем seed
df['baseline_prediction_1.75'] = df['description_text_stem'].apply(lambda descr: baseline_category(
    stem_description=descr,
    filtered_n_grams=filtered_n_grams_,
    total_points_cat=total_points_cat,
    cat_probabilities=CAT_PROBABILITIES,
    categories=CATEGORIES
    )
)

In [414]:
print('baseline accuracy (threshold 1.75):', (df['baseline_prediction_1.75'] == df['attr_value_name']).mean())

baseline accuracy (threshold 1.75): 0.3710119190946304


Точность упала

#### Пробую подобрать более оптимальный трешхолд

#### 1

In [415]:
filtered_n_grams_ = filter_n_grams(
    n_grams=n_grams,
    threshold=1, # задаем меньший трешхолд
    categories=CATEGORIES
)

total_points_cat = calc_total_points(
    filtered_n_grams=filtered_n_grams_,
    categories=CATEGORIES
)

np.random.seed(RANDOM_STATE) # обязательно фиксируем seed
df['baseline_prediction_1'] = df['description_text_stem'].apply(lambda descr: baseline_category(
    stem_description=descr,
    filtered_n_grams=filtered_n_grams_,
    total_points_cat=total_points_cat,
    cat_probabilities=CAT_PROBABILITIES,
    categories=CATEGORIES
    )
)

In [416]:
print('baseline accuracy (threshold 1):', (df['baseline_prediction_1'] == df['attr_value_name']).mean())

baseline accuracy (threshold 1): 0.4867264627979774


Ого, тут все ещё лучше.

И это даже лучше, чем всегда выдавать самую частую категорию (`косметический`)

#### 1.125

In [418]:
filtered_n_grams_ = filter_n_grams(
    n_grams=n_grams,
    threshold=1.125, # задаем меньший трешхолд
    categories=CATEGORIES
)

total_points_cat = calc_total_points(
    filtered_n_grams=filtered_n_grams_,
    categories=CATEGORIES
)

np.random.seed(RANDOM_STATE) # обязательно фиксируем seed
df['baseline_prediction_1.125'] = df['description_text_stem'].apply(lambda descr: baseline_category(
    stem_description=descr,
    filtered_n_grams=filtered_n_grams_,
    total_points_cat=total_points_cat,
    cat_probabilities=CAT_PROBABILITIES,
    categories=CATEGORIES
    )
)

In [419]:
print('baseline accuracy (threshold 1.125):', (df['baseline_prediction_1.125'] == df['attr_value_name']).mean())

baseline accuracy (threshold 1.125): 0.42760755277309576


Тоже все грустно

#### Ну и последняя попытка..

In [420]:
filtered_n_grams_ = filter_n_grams(
    n_grams=n_grams,
    threshold=1.05, # задаем меньший трешхолд
    categories=CATEGORIES
)

total_points_cat = calc_total_points(
    filtered_n_grams=filtered_n_grams_,
    categories=CATEGORIES
)

np.random.seed(RANDOM_STATE) # обязательно фиксируем seed
df['baseline_prediction_1.05'] = df['description_text_stem'].apply(lambda descr: baseline_category(
    stem_description=descr,
    filtered_n_grams=filtered_n_grams_,
    total_points_cat=total_points_cat,
    cat_probabilities=CAT_PROBABILITIES,
    categories=CATEGORIES
    )
)

In [421]:
print('baseline accuracy (threshold 1.05):', (df['baseline_prediction_1.05'] == df['attr_value_name']).mean())

baseline accuracy (threshold 1.05): 0.4405550204671322


# Фиксируем прометку

**Итог**: Берем базелин с трешхолдом 1

In [423]:
df[['item_id', 'baseline_prediction_1']].rename(
    columns={'baseline_prediction_1': 'baseline_prediction'}
    ).to_csv('baseline_prediction.csv', index=False)

In [430]:
print('baseline accuracy (threshold 1):', (df['baseline_prediction_1'] == df['attr_value_name']).mean())

baseline accuracy (threshold 1): 0.4867264627979774
